In [6]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import random

In [7]:
def create_query_gate(n, is_constant):
    """
    Creates a quantum circuit implementing the query gate for a given function.
    :param n: Number of input qubits
    :param is_constant: Boolean indicating whether the function is constant
    :return: A QuantumCircuit representing the query gate
    """
    qc = QuantumCircuit(n + 1)
    if is_constant:
        # Constant function: flip the ancilla qubit if f(x) = 1
        if random.choice([0, 1]) == 1:
            qc.x(n)
    else:
        # Balanced function: flip the ancilla qubit for half the inputs
        for i in range(n):
            qc.cx(i, n)
    return qc


In [8]:
def deutsch_jozsa_circuit(query_gate, n):
    """
    Implements the Deutsch-Jozsa algorithm.
    :param query_gate: The query gate as a QuantumCircuit
    :param n: Number of input qubits
    :return: The complete Deutsch-Jozsa circuit
    """
    dj_circuit = QuantumCircuit(n + 1, n)
    
    # Initialize the ancilla qubit in |1> and apply Hadamard gates
    dj_circuit.x(n)
    dj_circuit.h(range(n + 1))
    
    # Append the query gate
    dj_circuit.append(query_gate.to_gate(), range(n + 1))
    
    # Apply Hadamard gates to the input qubits
    dj_circuit.h(range(n))
    
    # Measure the input qubits
    dj_circuit.measure(range(n), range(n))
    return dj_circuit


In [9]:
# Number of input qubits
n = 3

# Randomly decide if the function is constant or balanced
is_constant = random.choice([True, False])

# Generate the query gate
query_gate = create_query_gate(n, is_constant)

# Print the query gate circuit
print("Query Gate Circuit:")
print(query_gate.draw())

# Create the Deutsch-Jozsa circuit
dj_circuit = deutsch_jozsa_circuit(query_gate, n)

# Print the Deutsch-Jozsa circuit
print("\nDeutsch-Jozsa Circuit:")
print(dj_circuit.draw())

# Simulate the circuit
simulator = Aer.get_backend('qasm_simulator')
result = simulator.run(dj_circuit, shots=1024).result()
counts = result.get_counts()

# Display the results
print("\nMeasurement Results:")
print(counts)
plot_histogram(counts).show()

# Determine the type of function based on measurement
if '0' * n in counts:
    print("\nThe function is constant.")
else:
    print("\nThe function is balanced.")


Query Gate Circuit:
     
q_0: 
     
q_1: 
     
q_2: 
     
q_3: 
     

Deutsch-Jozsa Circuit:
     ┌───┐     ┌──────────────┐┌───┐┌─┐      
q_0: ┤ H ├─────┤0             ├┤ H ├┤M├──────
     ├───┤     │              │├───┤└╥┘┌─┐   
q_1: ┤ H ├─────┤1             ├┤ H ├─╫─┤M├───
     ├───┤     │  circuit-171 │├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├─────┤2             ├┤ H ├─╫──╫─┤M├
     ├───┤┌───┐│              │└───┘ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├┤3             ├──────╫──╫──╫─
     └───┘└───┘└──────────────┘      ║  ║  ║ 
c: 3/════════════════════════════════╩══╩══╩═
                                     0  1  2 


AerError: 'unknown instruction: circuit-171'